In [40]:
import pandas as pd
import numpy as np 
import matplotlib as mpl
import seaborn as sns

In [41]:
FEATURE = "Corruption_Perception_Index"
df = pd.read_csv("..\..\datasets\Original\%s.csv" % FEATURE)
df

,Country,ISO3,Region,CPI score 2020,Rank 2020,Sources 2020,Standard error 2020,CPI score 2019,Rank 2019,Sources 2019,...,Rank 2018,Sources 2018,Standard error 2018,CPI score 2017,Rank 2017,Sources 2017,Standard error 2017,CPI score 2016,Sources 2016,Standard error 2016
0,Denmark,DNK,WE/EU,88,1,8,1.78,87,1,8,...,1,8,2.63,88,2,8,2.75,90.0,7.0,2.46
1,New Zealand,NZL,AP,88,1,8,1.48,87,1,8,...,2,8,2.44,89,1,8,2.40,90.0,7.0,2.56
2,Finland,FIN,WE/EU,85,3,8,1.75,86,3,8,...,3,8,2.74,85,3,8,2.84,89.0,7.0,1.46
3,Singapore,SGP,AP,85,3,9,1.20,85,4,9,...,3,9,1.98,84,6,9,2.26,84.0,8.0,2.35
4,Sweden,SWE,WE/EU,85,3,8,1.30,85,4,8,...,3,8,2.02,84,6,8,2.27,88.0,7.0,1.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Venezuela,VEN,AME,15,176,8,0.91,16,173,8,...,168,8,2.13,18,169,8,1.60,17.0,7.0,1.41
176,Yemen,YEM,MENA,15,176,7,1.33,15,177,7,...,176,7,1.45,16,175,7,1.85,14.0,7.0,3.05
177,Syria,SYR,MENA,14,178,5,1.58,13,178,5,...,178,5,1.92,14,178,5,1.93,13.0,5.0,1.97
178,Somalia,SOM,SSA,12,179,6,2.29,9,180,5,...,180,6,3.08,9,180,5,2.26,10.0,5.0,2.98


## Skeleton dataframe

In [42]:
columns = ["ISO3"]
df.drop_duplicates(subset=columns, keep="first")
df_new = df[columns[0]].unique()
df_new = pd.DataFrame(df_new, columns=columns)
df.rename(columns={"ISO3": "ISO3_Code"}, inplace=True)
df_new.rename(columns={"ISO3": "ISO3_Code"}, inplace=True)
df_new.shape

(180, 1)

In [43]:
df_code = pd.read_csv("../../datasets/Original/AreaCode_ISO3Code_conversion.csv")
df_code = df_code[["Country", "ISO3 Code", "M49 Code"]]
df_new = df_new.join(df_code.set_index("ISO3 Code"), on="ISO3_Code", validate="1:1")
df_new

,ISO3_Code,Country,M49 Code
0,DNK,Denmark,208.0
1,NZL,New Zealand,554.0
2,FIN,Finland,246.0
3,SGP,Singapore,702.0
4,SWE,Sweden,752.0
...,...,...,...
175,VEN,Venezuela (Bolivarian Republic of),862.0
176,YEM,Yemen,887.0
177,SYR,Syrian Arab Republic,760.0
178,SOM,Somalia,706.0


In [44]:
df_new.rename(columns={"ISO3 Code": "ISO3_Code", "Country": "Country_Name", "M49 Code": "M49_Code"}, inplace=True)
df_new.head()

,ISO3_Code,Country_Name,M49_Code
0,DNK,Denmark,208.0
1,NZL,New Zealand,554.0
2,FIN,Finland,246.0
3,SGP,Singapore,702.0
4,SWE,Sweden,752.0


In [45]:
df_new[df_new["Country_Name"].isnull()]

,ISO3_Code,Country_Name,M49_Code
107,KSV,NaN,NaN


In [46]:
# drop the null rows cause it's the legend
df_new.dropna(inplace=True)
df_new

,ISO3_Code,Country_Name,M49_Code
0,DNK,Denmark,208.0
1,NZL,New Zealand,554.0
2,FIN,Finland,246.0
3,SGP,Singapore,702.0
4,SWE,Sweden,752.0
...,...,...,...
175,VEN,Venezuela (Bolivarian Republic of),862.0
176,YEM,Yemen,887.0
177,SYR,Syrian Arab Republic,760.0
178,SOM,Somalia,706.0


## Unemployment
Unemployment (% of labor force)

In [47]:

df_tmp = pd.concat([df.loc[:, ["ISO3_Code"]], df.loc[:, df.columns[df.columns.str.contains("CPI score")]]], axis=1)
df_new = df_new.merge(df_tmp, on="ISO3_Code", how="inner")
columns = df.columns[df.columns.str.contains("CPI score")].values
names_dict = {}
for i, column in enumerate(columns):
    names_dict[column] = "%s_%s" % (FEATURE, column[-4:])
df_new.rename(columns=names_dict, inplace=True)
df_new

,ISO3_Code,Country_Name,M49_Code,Corruption_Perception_Index_2020,Corruption_Perception_Index_2019,Corruption_Perception_Index_2018,Corruption_Perception_Index_2017,Corruption_Perception_Index_2016
0,DNK,Denmark,208.0,88,87,88,88,90.0
1,NZL,New Zealand,554.0,88,87,87,89,90.0
2,FIN,Finland,246.0,85,86,85,85,89.0
3,SGP,Singapore,702.0,85,85,85,84,84.0
4,SWE,Sweden,752.0,85,85,85,84,88.0
...,...,...,...,...,...,...,...,...
174,VEN,Venezuela (Bolivarian Republic of),862.0,15,16,18,18,17.0
175,YEM,Yemen,887.0,15,15,14,16,14.0
176,SYR,Syrian Arab Republic,760.0,14,13,13,14,13.0
177,SOM,Somalia,706.0,12,9,10,9,10.0


In [48]:
# add a column for average over the years
columns = df_new.columns[-5:]
df_cols = df_new[columns].applymap(lambda x: float(x) if x not in ["", None, ".."] else None)
df_new = pd.concat([df_new.iloc[:, :3], df_cols], axis=1)
df_new["%s_Avg" % FEATURE] = df_new[columns].mean(axis=1)
df_new.head()

,ISO3_Code,Country_Name,M49_Code,Corruption_Perception_Index_2020,Corruption_Perception_Index_2019,Corruption_Perception_Index_2018,Corruption_Perception_Index_2017,Corruption_Perception_Index_2016,Corruption_Perception_Index_Avg
0,DNK,Denmark,208.0,88.0,87.0,88.0,88.0,90.0,88.2
1,NZL,New Zealand,554.0,88.0,87.0,87.0,89.0,90.0,88.2
2,FIN,Finland,246.0,85.0,86.0,85.0,85.0,89.0,86.0
3,SGP,Singapore,702.0,85.0,85.0,85.0,84.0,84.0,84.6
4,SWE,Sweden,752.0,85.0,85.0,85.0,84.0,88.0,85.4


In [49]:
# prettify the float
#df_new = df_new[df_new.columns[-6:]].applymap(na_action='ignore', func=lambda x: ".6f" % x)
with open("../../datasets/Processed/%s_Processed.csv" % FEATURE, "w", encoding="utf8") as f:
    df_new.to_csv(f, lineterminator="\n", index=False)

## Stats

In [50]:
df_new.isnull().sum()

ISO3_Code                           0
Country_Name                        0
M49_Code                            0
Corruption_Perception_Index_2020    0
Corruption_Perception_Index_2019    0
Corruption_Perception_Index_2018    0
Corruption_Perception_Index_2017    0
Corruption_Perception_Index_2016    4
Corruption_Perception_Index_Avg     0
dtype: int64